# Morningstar Pro - Entraînement Avancé sur Colab
## Système complet de trading algorithmique

In [ ]:
# Installation optimisée (version spécifique pour compatibilité)
%pip install -q tensorflow==2.12.0 pandas==1.5.3 numpy==1.23.5 ccxt==4.1.91 ta-lib pyarrow
%load_ext tensorboard
import sys
!{sys.executable} -m pip show tensorflow pandas numpy ccxt | grep Version
!git clone https://github.com/Cabrel10/eva001.git
%cd eva001

# Configuration matérielle
import tensorflow as tf
try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver.connect()
    strategy = tf.distribute.TPUStrategy(tpu)
    print('TPU détectée:', tpu.master())
except ValueError:
    strategy = tf.distribute.MirroredStrategy()
    print('Utilisation de', strategy.num_replicas_in_sync, 'GPU' if strategy.num_replicas_in_sync>1 else 'GPU')

In [ ]:
# Pipeline de données avancé
from Morningstar.utils.data_manager import ExchangeDataManager
from Morningstar.scripts.prepare_full_dataset import main as prepare_dataset
import pandas as pd

# Téléchargement multi-paires
def fetch_data(pairs, timeframe='1d', limit=2000):
    exchange = ExchangeDataManager(exchange="binance")
    all_data = []
    for pair in pairs:
        print(f"Téléchargement {pair}...")
        data = exchange.download_historical_data(pair, timeframe, limit)
        data['pair'] = pair
        all_data.append(data)
    return pd.concat(all_data)

# Paires majeures
pairs = ["BTC/USDT", "ETH/USDT", "BNB/USDT", "SOL/USDT"]
raw_data = fetch_data(pairs)

# Préprocessing
data = prepare_dataset(raw_data)
data.to_parquet('full_dataset.parquet')
print(f"Dataset final: {data.shape}")

In [ ]:
# Architecture avancée avec stratégie de distribution
with strategy.scope():
    inputs = tf.keras.Input(shape=(50, len(data.columns)))
    x = tf.keras.layers.Conv1D(128, 5, activation='swish')(inputs)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.LSTM(256, return_sequences=True)(x)
    x = tf.keras.layers.LSTM(128)(x)
    x = tf.keras.layers.Dense(64, activation='swish')(x)
    outputs = tf.keras.layers.Dense(1)(x)
    
    model = tf.keras.Model(inputs, outputs)
    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
        loss='huber',
        metrics=['mae']
    )

In [ ]:
# Entraînement avec callbacks avancés
callbacks = [
    tf.keras.callbacks.ModelCheckpoint('best_model.h5', save_best_only=True),
    tf.keras.callbacks.ReduceLROnPlateau(factor=0.5, patience=5),
    tf.keras.callbacks.TensorBoard(log_dir='./logs')
]

history = model.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=200,
    batch_size=1024,
    callbacks=callbacks
)

In [ ]:
# Analyse approfondie
!tensorboard dev upload --logdir ./logs \
  --name "Morningstar Training" \
  --description "Résultats d'entraînement"

# Sauvegarde finale
model.save('morningstar_pro.h5')
!cp *.h5 '/content/drive/MyDrive/Colab Data/'